Load the environment variables from a .env file

To get an api key, navigate to https://aistudio.google.com/apikey to create one. Then create a .env file in the same directory as this notebook with the following content:
```GOOGLE_API_KEY=your_api_key_here```


In [25]:
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.getenv("GOOGLE_API_KEY")

Single shot question answering with Gemini 2.0 Flash to check correct working of the code

In [26]:
from google import genai
from pydantic import BaseModel

class Recipe(BaseModel):
    medicijn: str
    categorie: str
    recept: str
    medische_vraag: str

client = genai.Client(api_key=api_key)
response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents="Ik heb een infectie waarvoor ik een antibioticum nodig heb. Welk middel kan ik gebruiken?",
    config={
        "response_mime_type": "application/json",
        "response_schema": Recipe,
        "system_instruction": "Formuleer het recept volgens template 'Rp. [Medicijnnaam] [Sterkte] [Vorm]\ndtd. No. [Aantal]\n S. [Instructie]'. Geef bij de instructie ook de duur van de behandeling aan.",
    },
)

print(response.text)

{
  "medicijn": "Amoxicilline",
  "categorie": "Antibioticum",
  "recept": "Rp. Amoxicilline 500mg capsules\ndtd. No. 21\nS. driemaal daags 1 capsule gedurende 7 dagen.",
  "medische_vraag": "Ik heb een infectie waarvoor ik een antibioticum nodig heb. Welk middel kan ik gebruiken?"
}


Create a function to get a recipe based on a medical question

In [27]:
def get_recipe(medical_question: str) -> Recipe:
    response = client.models.generate_content(
        model="gemini-2.0-flash",
        contents=medical_question,
        config={
            "response_mime_type": "application/json",
            "response_schema": Recipe,
            "system_instruction": "Formuleer het recept volgens template 'Rp. [Medicijnnaam] [Sterkte] [Vorm]\ndtd. No. [Aantal]\n S. [Instructie]'. Geef bij de instructie ook de duur van de behandeling aan.",
        },
    )
    return response.parsed

recipe = get_recipe("Ik heb last van brandend maagzuur of zure reflux. Wat kan ik hiervoor nemen?")
print(recipe.model_dump_json())


{"medicijn":"Omeprazol","categorie":"Maagbeschermer","recept":"Rp. Omeprazol 20mg capsule\ndtd. No. 30\nS. 1 capsule per dag gedurende 4 weken.","medische_vraag":"Ik heb last van brandend maagzuur of zure reflux. Wat kan ik hiervoor nemen?"}


In [31]:
import pandas as pd
filepath = "../data/example_recipes_with_question.json"
records = pd.read_json(filepath)

In [35]:
recipes = []
for index, row in records.head(10).iterrows():
    question = row["medische-vraag"]
    recipes.append(get_recipe(question))



KeyboardInterrupt: 